<a href="https://colab.research.google.com/github/Songtuan/COMP6340-Assignment-1/blob/master/Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import numpy as np
import unittest
from torch.autograd import Variable

In [15]:
class NetWork(nn.Module):
    def __init__(self):
      super(NetWork, self).__init__()
      self.conv = nn.ConvTranspose2d(64, 32, 5, 2, 0)
    def forward(self, x):
      output = self.conv(x)
      return output
    
network = NetWork()
output = network(torch.rand(32, 64, 31, 31))
print(output.shape)

torch.Size([32, 32, 65, 65])


## Question 1
### (a)
The output will has the size $32 \times 16 \times 64 \times 64$.
### (b)
The output will has the size $32 \times 128 \times 3 \times 3$.
### (c)
The output will has the size $32 \times 32 \times 65 \times 65$.
### (d)
There are 432 learnable parameters.

## Question 2

In [0]:
class Encoder(nn.Module):
    """Encoder network to map from an RGB image to a latent feature vector."""

    def __init__(self, z_dim=64, img_size=64):
        super(Encoder, self).__init__()

        self.z_dim = z_dim
        self.hidden_layer = nn.Sequential(nn.Linear(3 * img_size * img_size, self.z_dim * 1 * 1), 
                                         nn.BatchNorm1d(self.z_dim),
                                         nn.ReLU(True)
                                         )
        self.output_layer = nn.Sequential(nn.Linear(self.z_dim * 1 * 1, self.z_dim * 1 * 1),
                                          nn.Tanh()
                                         )

        ####################################################################
        # TODO: Create a nn.Sequential model for each layer in the encoder #
        # as described in the assignment specification and assign them to  #
        # self.hidden_layer and self.output_layer.                         #
        ####################################################################



    def forward(self, x):
        x = x.view(x.size()[0], -1)
        x = self.output_layer(self.hidden_layer(x))
        return x

In [0]:
class Decoder(nn.Module):
    """Decoder network to map from a latent feature vector to an RGB image."""

    def __init__(self, z_dim=64, img_size=64):
        super(Decoder, self).__init__()

        assert img_size==64
        self.z_dim = z_dim
        self.layer1 = nn.Sequential(nn.ConvTranspose2d(self.z_dim, 128, 7, 1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(True),
                                    nn.Conv2d(128, 128, 3, 1, 1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(True)
                                   )
        self.layer2 = nn.Sequential(nn.ConvTranspose2d(128, 64, 3, 2),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True),
                                    nn.Conv2d(64, 64, 3, 1, 1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True)
                                   )
        self.layer3 = nn.Sequential(nn.ConvTranspose2d(64, 64, 3, 2),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True),
                                    nn.Conv2d(64, 64, 3, 1, 1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True)
                                   )
        self.layer4 = nn.Sequential(nn.ConvTranspose2d(64, 32, 3, 2),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(True),
                                    nn.Conv2d(32, 32, 3, 1, 1),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(True)
                                   )
        self.layer5 = nn.Sequential(nn.ConvTranspose2d(32, 32, 2, 1),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(True),
                                    nn.Conv2d(32, 3, 1, 1),
                                    nn.Tanh()
                                   )

        ####################################################################
        # TODO: Create a nn.Sequential model for each layer in the decoder #
        # as described in the assignment specification. Assign them to     #
        # self.layer1, self.layer2, etc. The test_models.py script will    #
        # help you check that you got the layers correct.                  #
        ####################################################################



    def forward(self, x):
        x = x.view(x.size()[0], self.z_dim, 1, 1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return x

In [0]:
class TestEncoder(unittest.TestCase):

    def test_size(self):
        """Test that the Encoder produces the correct size output."""
        batch_size = 32
        image_size = 64
        latent_size = 128
        
        E = Encoder(latent_size, image_size)
        tensor_in = Variable(2.0 * (torch.rand((batch_size, 3, image_size, image_size)) - 0.5))
        tensor_out = E.forward(tensor_in)
        self.assertEqual(tensor_out.size(), torch.Size([batch_size, latent_size]))


    def test_layers(self):
        """Test that the Encoder produces the correct size output for each layer."""
        batch_size = 32
        image_size = 64
        latent_size = 128

        E = Encoder(latent_size, image_size)
        tensor = Variable(2.0 * (torch.rand((batch_size, 3, image_size, image_size)) - 0.5))
        tensor = tensor.view(tensor.size()[0], -1)
        tensor = E.hidden_layer(tensor)
        self.assertEqual(tensor.size(), torch.Size([batch_size, latent_size]))
        tensor = E.output_layer(tensor)
        self.assertEqual(tensor.size(), torch.Size([batch_size, latent_size]))

In [0]:
class TestDecoder(unittest.TestCase):
    
    def test_size(self):
        """Test that the Decoder produced the correct size output."""
        batch_size = 32
        image_size = 64
        latent_size = 128

        D = Decoder(latent_size, image_size)
        tensor_in = Variable(2.0 * (torch.rand((batch_size, latent_size, 1, 1)) - 0.5))
        tensor_out = D.forward(tensor_in)
        self.assertEqual(tensor_out.size(), torch.Size([batch_size, 3, image_size, image_size]))


    def test_layers(self):
        """Test that the Decoder produced the correct size output for each layer."""
        batch_size = 32
        image_size = 64
        latent_size = 128

        D = Decoder(latent_size, image_size)
        tensor = Variable(2.0 * (torch.rand((batch_size, latent_size, 1, 1)) - 0.5))
        tensor = D.layer1.forward(tensor)
        self.assertEqual(tensor.size(), torch.Size([batch_size, 128, 7, 7]))
        tensor = D.layer2.forward(tensor)
        self.assertEqual(tensor.size(), torch.Size([batch_size, 64, 15, 15]))
        tensor = D.layer3.forward(tensor)
        self.assertEqual(tensor.size(), torch.Size([batch_size, 64, 31, 31]))
        tensor = D.layer4.forward(tensor)
        self.assertEqual(tensor.size(), torch.Size([batch_size, 32, 63, 63]))
        tensor = D.layer5.forward(tensor)
        self.assertEqual(tensor.size(), torch.Size([batch_size, 3, image_size, image_size]))


In [7]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 1.106s

OK


In [9]:
SEED = 4680

np.random.seed(SEED)
torch.manual_seed(SEED)

In [15]:
from google.colab import files
files.upload()

Saving data.zip to data.zip


{'data.zip': b'PK\x03\x04\n\x00\x00\x00\x00\x00\xee\x894M\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00data/PK\x03\x04\n\x00\x00\x00\x00\x00\xb5c\xbcN\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00data/1/PK\x03\x04\x14\x00\x00\x00\x08\x00\xee\x894M\xb8T\x91\xc8\xf9\x13\x00\x000\x14\x00\x00\x0e\x00\x00\x00data/1/111.png-\x98{<S\x7f\xfc\xc7\x87an\xb9$\x91\xcb\xdcC\xc2\x92Ks\x19s\x89\xc9]\xdfj\xee\x96\xa4\x85X\xae+\xd7\xe5\xd6\xdc\xb2\x10K\xa2\xb96\xf7!\xac\xdc\x86\xdc\x8b\x88r-s\xcb=w\xe5\xb7~\xbf\xdf\xeb\xf1:\xcf\xcf\xeb\xbc\xdf\xef\xcf\xe7\xfcs\xfe\xfa$\xd8X]\xe5\xe1<\xc7\t\x00\x00x\xcc\xcd\x8c\xed\x18k\xe4\xbf\x07\xc4\xc2\xe0\xef\x99\xb4\x03\xc6\xc2\x82\x81\x9b;\xb0\xb2\xb2\n\xcbO\x19\x01\x00\xccOm\xae9\x98\x00N\x18\r\x80\x08\x1f\x9f\x08\x9f\x08CR"R\xff+E)E\x88\xe2?A\x14!\x10\x88\xc8\xff-\x0cA!\x108\x04\n\x85@\x11\x10\xa8\x1dT\xf1_\x82B\xe0\xd0\x7fb\x10\x01e\x14\xe1p(\x1c\x01\x85#\xa1p\x14\x1c\x04\x80+B\xe1\xffJ\x08\xf8?1h\x07G \xe0\x08$\x1c\x81\x82#

In [16]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/1/
  inflating: data/1/111.png          
  inflating: data/1/112.png          
  inflating: data/1/123.png          
  inflating: data/1/129.png          
  inflating: data/1/130.png          
  inflating: data/1/131.png          
  inflating: data/1/133.png          
  inflating: data/1/134.png          
  inflating: data/1/135.png          
  inflating: data/1/136.png          
  inflating: data/1/143.png          
  inflating: data/1/144.png          
  inflating: data/1/147.png          
  inflating: data/1/148.png          
  inflating: data/1/149.png          
  inflating: data/1/15.png           
  inflating: data/1/150.png          
  inflating: data/1/169.png          
  inflating: data/1/172.png          
  inflating: data/1/174.png          
  inflating: data/1/179.png          
  inflating: data/1/180.png          
  inflating: data/1/181.png          
  inflating: data/1/194.png          
  inflating: data/1/195.png  

In [0]:
RUN_ON_GPU = torch.cuda.is_available()
if RUN_ON_GPU:
    torch.cuda.manual_seed(SEED)


def np_to_var(x):
    """Converts numpy to variable."""
    if RUN_ON_GPU:
        x = x.cuda()
    return Variable(x)


def var_to_np(x):
    """Converts variable to numpy."""
    if RUN_ON_GPU:
        x = x.cpu()
    return x.data.numpy()


def get_data_loader(image_size=64, batch_size=16):
    """Creates training data loader."""
    transform = transforms.Compose([
                    transforms.Resize(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])

    train_dataset = datasets.ImageFolder('./data/', transform)
    return DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [0]:
def create_models(image_size=64, latent_size=128):
    """Creates encoder and decoder and moves them to the GPU if requested."""

    E = Encoder(latent_size, image_size)
    D = Decoder(latent_size, image_size)

    if RUN_ON_GPU:
        print('Moving models to GPU.')
        E.cuda()
        D.cuda()
    else:
        print('Keeping models on CPU.')

    return E, D


def train_models(E, D, image_size=64, latent_size=128, batch_size=32, num_epochs=500,
                 log_interval=10, save_interval=100, out_dir='samples'):
    """Train models."""

    # create optimizers
    e_optimizer = optim.Adam(E.parameters(), 1.0e-3, [0.5, 0.999])
    d_optimizer = optim.Adam(D.parameters(), 2.0e-3, [0.5, 0.999])

    # create output directory for image samples
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    dataloader = get_data_loader(image_size=image_size, batch_size=batch_size)
    total_train_iters = num_epochs * len(dataloader)

    loss_history = []
    print('Started training at {}'.format(time.asctime(time.localtime(time.time()))))

    # train for num_epochs
    for epoch in range(num_epochs):
        for iteration, batch in enumerate(dataloader, epoch * len(dataloader) + 1):

            real_images, _ = batch
            noisy_images = torch.clamp(torch.add(real_images, 2.0 * (torch.rand(real_images.size()) - 0.5)), min=-1.0, max=1.0)
            real_images = np_to_var(real_images)
            noisy_images = np_to_var(noisy_images)

            e_optimizer.zero_grad()
            d_optimizer.zero_grad()
            out_images = D.forward(E.forward(noisy_images))

            # compute reconstruction loss and update parameters
            loss = torch.mean((real_images - out_images).abs())
            loss.backward()
            e_optimizer.step()
            d_optimizer.step()

            # print the log info
            if iteration % log_interval == 0:
                print('Iteration [{:6d}/{:6d}] | loss: {:.4f}'.format(
                    iteration, total_train_iters, loss.data.item()))

            # keep track of loss for plotting and saving
            loss_history.append(loss.data.item())

            # save the generated samples and loss
            if iteration % save_interval == 0:
                path = os.path.join(out_dir, 'sample-{:06d}.png'.format(iteration))
                torchvision.utils.save_image(torch.cat((real_images, noisy_images, out_images)),
                                             path, nrow=real_images.size()[0], normalize=True)
                print('Saved {}'.format(path))

                # save the loss history
                with open('loss.txt', 'wt') as file:
                    file.write('\n'.join(['{}'.format(loss) for loss in loss_history]))
                    file.write('\n')

                # generate and save some novel images
                noise = np_to_var(2.0 * (torch.rand((batch_size, latent_size, 1, 1)) - 0.5))
                novel_images = D.forward(noise)
                path = os.path.join(out_dir, 'novel-{:06d}.png'.format(iteration))
                torchvision.utils.save_image(novel_images, path, normalize=True)
                print('Saved {}'.format(path))



In [21]:
import os
import argparse
import numpy as np
import time

import torch
import torch.optim as optim
from torch.autograd import Variable
import torchvision.utils

from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

def main():
    # set up commandline arguments
#     parser = argparse.ArgumentParser(description='COMP4680/8650 Assignment 6')
#     parser.add_argument('--image-size', type=int, default=64, metavar='N',
#                         help='image dimensions (width and height) (default: 64)')
#     parser.add_argument('--latent-size', type=int, default=128, metavar='N',
#                         help='latent representation size (default: 128)')
#     parser.add_argument('--batch-size', type=int, default=32, metavar='N',
#                         help='batch size for training (default: 32)')
#     parser.add_argument('--epochs', type=int, default=500, metavar='N',
#                         help='number of epochs to train (default: 500)')
#     parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                         help='iterations to wait between printing progress (default: 10)')
#     parser.add_argument('--save-interval', type=int, default=100, metavar='N',
#                         help='iterations to wait between saving samples (default: 100)')
#     parser.add_argument('--output-dir', type=str, default='samples', metavar='DIR',
#                         help='output directory for image samples')

#     args = parser.parse_args()
 

    # create and train models
    E, D = create_models(64, 128)
    train_models(E, D, image_size=64, latent_size=128,
                 batch_size=32, num_epochs=500,
                 log_interval=10, save_interval=100,
                 out_dir='samples')


if __name__ == '__main__':
    main()

Moving models to GPU.
Started training at Tue May 28 06:12:45 2019
Iteration [    10/  3500] | loss: 0.5164
Iteration [    20/  3500] | loss: 0.3807
Iteration [    30/  3500] | loss: 0.3179
Iteration [    40/  3500] | loss: 0.2765
Iteration [    50/  3500] | loss: 0.2383
Iteration [    60/  3500] | loss: 0.2058
Iteration [    70/  3500] | loss: 0.2045
Iteration [    80/  3500] | loss: 0.1863
Iteration [    90/  3500] | loss: 0.1838
Iteration [   100/  3500] | loss: 0.1610
Saved samples/sample-000100.png
Saved samples/novel-000100.png
Iteration [   110/  3500] | loss: 0.1698
Iteration [   120/  3500] | loss: 0.1564
Iteration [   130/  3500] | loss: 0.1391
Iteration [   140/  3500] | loss: 0.1333
Iteration [   150/  3500] | loss: 0.1376
Iteration [   160/  3500] | loss: 0.1339
Iteration [   170/  3500] | loss: 0.1414
Iteration [   180/  3500] | loss: 0.1256
Iteration [   190/  3500] | loss: 0.1402
Iteration [   200/  3500] | loss: 0.1209
Saved samples/sample-000200.png
Saved samples/nove

In [23]:
!zip -r samples.zip samples

  adding: samples/ (stored 0%)
  adding: samples/novel-002100.png (deflated 0%)
  adding: samples/sample-000800.png (deflated 0%)
  adding: samples/sample-002100.png (deflated 0%)
  adding: samples/sample-001300.png (deflated 0%)
  adding: samples/novel-001800.png (deflated 0%)
  adding: samples/novel-002500.png (deflated 0%)
  adding: samples/sample-003100.png (deflated 0%)
  adding: samples/novel-002400.png (deflated 0%)
  adding: samples/sample-003000.png (deflated 0%)
  adding: samples/novel-001900.png (deflated 0%)
  adding: samples/novel-002600.png (deflated 0%)
  adding: samples/novel-003200.png (deflated 0%)
  adding: samples/sample-002700.png (deflated 0%)
  adding: samples/sample-001900.png (deflated 0%)
  adding: samples/sample-000300.png (deflated 0%)
  adding: samples/sample-002800.png (deflated 0%)
  adding: samples/novel-001200.png (deflated 0%)
  adding: samples/sample-002000.png (deflated 0%)
  adding: samples/novel-000600.png (deflated 0%)
  adding: samples/sample-002